# Tuning GNN-ePC-SAFT

## Starting point

In [ ]:
import os, shutil
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("awskeyid")
secret_value_1 = user_secrets.get_secret("awssecretkey")
secret_value_2 = user_secrets.get_secret("ghtk")
secret_value_3 = user_secrets.get_secret("kaggle")
secret_value_4 = user_secrets.get_secret("username")
secret_value_5 = user_secrets.get_secret("wandbkey")
secret_value_6 = user_secrets.get_secret("email")

os.environ["KAGGLE_USERNAME"] = secret_value_4
os.environ["KAGGLE_KEY"] = secret_value_3
os.environ["WANDB_API_KEY"] = secret_value_5
os.environ["AWS_ACCESS_KEY_ID"] = secret_value_0
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_value_1

In [ ]:
# for the cloud 
!rm -rf .* *
!git clone --branch dev --single-branch -q https://{secret_value_2}@github.com/wildsonbbl/gnnepcsaft.git .

In [ ]:
!python -m pip install --upgrade pip

%pip install pipenv --user

import os 
os.environ['PATH'] = "/root/.local/bin:" + os.environ['PATH']

In [ ]:
!pipenv install wheel -q
!pipenv install -r requirements.txt -q

In [ ]:
# install pcsaft
!curl -O -L https://gitlab.com/libeigen/eigen/-/archive/master/eigen-master.zip
!curl -O -L https://github.com/zmeri/PC-SAFT/archive/refs/tags/v1.4.1.zip
!unzip -q eigen-master.zip
!unzip -q v1.4.1.zip
!cp -rf eigen-master/. PC-SAFT-1.4.1/externals/eigen
!pipenv install ./PC-SAFT-1.4.1 -q
# install pcsaft

In [ ]:
!echo AWS_ACCESS_KEY_ID={os.environ["AWS_ACCESS_KEY_ID"]} > .env
!echo AWS_SECRET_ACCESS_KEY={os.environ["AWS_SECRET_ACCESS_KEY"]} >>.env
!echo WANDB_API_KEY={os.environ["WANDB_API_KEY"]} >> .env

In [ ]:
!pipenv run dvc pull -f -q

In [ ]:
# previous running results
# !cp -a /kaggle/input/gnnepcsaft/tuning/root/* ~

## Tuning

### Tuning run

In [ ]:
!pipenv run python -m gnnepcsaft.train.tuner --workdir=/kaggle/working/gnnepcsaft --config=gnnepcsaft/configs/default.py \
--dataset=ramirez --tags="batch 3" \
--num_cpu=3 --num_gpus=1 --num_init_gpus=1 \
--num_samples=50 --time_budget_s=39600 --verbose=0\
--config.num_train_steps=11000 --config.log_every_steps=1000 --config.model=PNAL --config.batch_size=512 \
--get_result=False #--restoredir=/root/ray_results/gnnpcsaft/

### Clear workspace

In [ ]:
!rm -rf .* *

### Zip results

In [ ]:
!rm -rf ~/ray*/gnn*/tune*/check* ~/ray*/gnn*/tune*/wandb
!zip -r tuning.zip ~/ray* -q